In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
}

url = 'https://www.ilboursa.com/analyses/synthese_fiches'
response = requests.get(url, headers=headers, timeout=30)

print(f"Status: {response.status_code}")

if response.status_code == 200:
    # Parse with BeautifulSoup to extract quote codes from <a> tags
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table
    table = soup.find('table', {'id': 'tabQuotes'})
    
    if table:
        data = []
        
        # Iterate through each row (skip header)
        for tr in table.find('tbody').find_all('tr'):
            tds = tr.find_all('td')
            if len(tds) >= 4:
                # Extract company name and quote code from <a> tag
                a_tag = tds[0].find('a')
                company_name = a_tag.get_text(strip=True) if a_tag else tds[0].get_text(strip=True)
                
                # Extract quote code from href (e.g., "conseil/ADWYA" → "ADWYA")
                href = a_tag.get('href', '') if a_tag else ''
                quote_code = href.split('/')[-1] if '/' in href else ''
                
                # Extract other columns
                consensus = tds[1].get_text(strip=True)
                potentiel = tds[2].get_text(strip=True)
                
                # Extract recommendation from img src
                img = tds[3].find('img')
                conseil_img = img.get('src', '') if img else ''
                conseil = conseil_img.split('/')[-1].replace('.png', '') if conseil_img else ''
                
                data.append({
                    'Quote_Code': quote_code,
                    'Company_Name': company_name,
                    'Consensus': consensus,
                    'Potentiel': potentiel,
                    'Conseil': conseil
                })
        
        # Create DataFrame
        df = pd.DataFrame(data)
        df["Publication_Date"] = pd.to_datetime('today').strftime('%Y-%m-%d')

        print(f"✅ Extracted {len(df)} companies")
        print(df.head(10))
    else:
        print("❌ Table not found")

Status: 200
✅ Extracted 86 companies
  Quote_Code          Company_Name Consensus Potentiel Conseil  \
0      ADWYA                 ADWYA                        f_4b   
1      AETEC                AETECH                        f_4b   
2         AL   AIR LIQUIDE TUNISIE                        f_3b   
3         AB             AMEN BANK  60,33 DT    +7.74%    f_4b   
4        AMS                   AMS                        f_3b   
5        ATB    ARAB TUNISIAN BANK                        f_5b   
6        ATL   ARAB TUNISIAN LEASE                        f_4b   
7      ARTES                 ARTES                        f_3b   
8      ASSAD                 ASSAD   4,07 DT   +27.08%    f_3b   
9      ASSMA  ASSURANCES MAGHREBIA                        f_4b   

  Publication_Date  
0       2026-02-07  
1       2026-02-07  
2       2026-02-07  
3       2026-02-07  
4       2026-02-07  
5       2026-02-07  
6       2026-02-07  
7       2026-02-07  
8       2026-02-07  
9       2026-02-07  


In [7]:
for quote in df['Quote_Code']:
    url = f'https://www.ilboursa.com/marches/secteur/{quote}'
    response = requests.get(url, headers=headers, timeout=30)
    soup = BeautifulSoup(response.text, 'html.parser')
    sector = soup.title.string
    sector = sector.split('secteur')[-1].split('à la bourse')[0].strip()
    df.loc[df['Quote_Code'] == quote, 'Sector'] = sector

In [8]:
try:
    existing_df = pd.read_csv('quotes_potential.csv')
    # Combine with new data and drop duplicates based on Quote_Code
    combined_df = pd.concat([existing_df, df], ignore_index=True)
    combined_df.drop_duplicates(keep='last',  inplace=True)
    df = combined_df
    print(f"🔄 Combined with existing data, total unique companies: {len(df)}")
except FileNotFoundError:
    print("📁 No existing CSV found, creating new one.")
# Save to CSV
df.to_csv('quotes_potential.csv', index=False)
print(f"\n💾 Saved to quotes_potential.csv")

📁 No existing CSV found, creating new one.

💾 Saved to quotes_potential.csv
